In [ ]:
! apt update

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InR

In [ ]:
#Java JDK
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Downloading Spark
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
#Unzipping the hadoop file
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

In [ ]:
###################### SPARK SETUP ################################
#Install findspark
!pip install -q findspark

In [ ]:
!pip install py4j

     |████████████████████████████████| 198 kB 22.5 MB/s 


In [ ]:
#Setting up environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [ ]:
#Initialize Spark session using findspark lib
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [ ]:
import os
import time

from pyspark.sql import SparkSession
import pyspark 

from pyspark.sql.functions import lit
from pyspark.ml.feature import IndexToString
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit,CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml import Pipeline, PipelineModel
import pyspark as ps

import math
import numpy as np
import pandas as pd


import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

spark = ps.sql.SparkSession.builder \
            .master("local[4]") \
            .appName("building recommender") \
            .getOrCreate() # create a spark session
            
sc = spark.sparkContext # create a spark context
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [ ]:
!pip install py4j

pre-processing

In [ ]:
#Unzip the file
!unzip book1M.zip

Archive:  book1M.zip
   creating: book1M/
  inflating: book1M/BX-Book-Ratings.csv  
  inflating: book1M/BX-Books.csv     
  inflating: book1M/BX-Users.csv     
  inflating: book1M/explicit_ratings_books.csv  


In [ ]:
# read ratings CSV
ratings_df = spark.read.csv('/content/book1M/explicit_ratings_books.csv',
                         header=True,       # use headers or not
                         quote='"',         # char for quotes
                         sep=",",           # char for separation
                         inferSchema=True)  # do we infer schema or not ?
ratings_df.printSchema()

root
 |-- userID: integer (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- author: string (nullable = true)
 |-- yearOfPublication: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- age: string (nullable = true)
 |-- country: string (nullable = true)



In [ ]:
# selecting the columns to work with in the dataset, we do not need all columns for the prediction, only userID, ISBN & rating column 
data=ratings_df.select(ratings_df['userID'],ratings_df['ISBN'],ratings_df['rating'])
data.show()

+------+----------+------+
|userID|      ISBN|rating|
+------+----------+------+
|276726|0155061224|     5|
|276729|052165615X|     3|
|276729|0521795028|     6|
|276744|038550120X|     7|
|276747|0060517794|     9|
|276747|0671537458|     9|
|276747|0679776818|     8|
|276747|0943066433|     7|
|276747|1885408226|     7|
|276748|0747558167|     6|
|276751|3596218098|     8|
|276754|0684867621|     8|
|276755|0451166892|     5|
|276762|0380711524|     5|
|276762|3453092007|     8|
|276772|0553572369|     7|
|276772|3499230933|    10|
|276772|3596151465|    10|
|276774|3442136644|     9|
|276786|8437606322|     8|
+------+----------+------+
only showing top 20 rows



In [ ]:
# Converting String columns (userID & ISBN) to index
s_indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in list(set(data.columns)-set(['rating'])) ]
pipeline = Pipeline(stages=s_indexer)
dftransform = pipeline.fit(data).transform(data)
dftransform.show()

+------+----------+------+------------+----------+
|userID|      ISBN|rating|userID_index|ISBN_index|
+------+----------+------+------------+----------+
|276726|0155061224|     5|     56337.0|   58420.0|
|276729|052165615X|     3|     26364.0|   87191.0|
|276729|0521795028|     6|     26364.0|   87208.0|
|276744|038550120X|     7|     56338.0|     216.0|
|276747|0060517794|     9|     12196.0|    1070.0|
|276747|0671537458|     9|     12196.0|    2592.0|
|276747|0679776818|     8|     12196.0|    1951.0|
|276747|0943066433|     7|     12196.0|  124008.0|
|276747|1885408226|     7|     12196.0|  137096.0|
|276748|0747558167|     6|     56339.0|   42440.0|
|276751|3596218098|     8|     56340.0|  145073.0|
|276754|0684867621|     8|     56341.0|     374.0|
|276755|0451166892|     5|     56342.0|     198.0|
|276762|0380711524|     5|     26365.0|    2698.0|
|276762|3453092007|     8|     26365.0|   27412.0|
|276772|0553572369|     7|     18853.0|    6882.0|
|276772|3499230933|    10|     

In [ ]:
ratings = dftransform.rdd

numRatings = ratings.count()
numUsers = ratings.map(lambda r: r[0]).distinct().count()
numbook = ratings.map(lambda r: r[1]).distinct().count()

print ("Got %d ratings from %d users on %d book." % (numRatings, numUsers, numbook))

KeyboardInterrupt: ignored

In [ ]:
book_counts = dftransform.groupBy(col("userID")).agg(F.count(col("rating")).alias("counts"))
book_counts.show()

NameError: ignored

eda

In [ ]:
ratings = ratings.toDF()
ratings.show()

KeyboardInterrupt: ignored

In [ ]:
ratings.describe().show()

Peringkat kenaikan .5 acara dipilih dengan frekuensi lebih sedikit daripada nilai bilangan bulat. Ini adalah indikator yang mungkin orang tidak tahu bahwa mereka dapat menilai dengan penambahan 0,5. Mungkin ini lebih menunjukkan masalah antarmuka dengan situs web daripada pendapat aktual pengguna tentang film

In [ ]:
ratingsRDD = dftransform.rdd.map(lambda p: Row(userID=int(p[0]), ISBN=int(p[1]),rating=float(p[2])))


In [ ]:
ratings_df.take(3)

KeyboardInterrupt: ignored

Pertama saya akan membagi data menjadi set pelatihan, validasi, dan pengujian

In [ ]:
training_df, validation_df, test_df =dftransform.randomSplit([0.8, 0.2, 0.2], seed=0)
# training_RDD = training_RDD.rdd.cache()
# validation_for_predict_RDD = validation_RDD.rdd.map(lambda x: (x[0], x[1])).cache()
# test_for_predict_RDD = test_RDD.rdd.map(lambda x: (x[0], x[1])).cache()
training_df

DataFrame[userID: int, ISBN: string, rating: int, userID_index: double, ISBN_index: double]

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml import Pipeline
from pyspark.sql import Row
import numpy as np
import math

Pertama saya akan mencoba alternating least squares estimator dengan parameter awal berikut. Saya akan mengulangi peringkat dan memilih model terbaik. Untuk pemeriksaan kesalahan saya menggunakan Root mean squared error.

In [ ]:
seed = 5
iterations = 10
regularization_parameter =0.1
ranks = range(4, 12)
errors = []
err = 0
tolerance = 0.02

In [ ]:
min_error = float('inf')
best_rank = -1
best_iteration = -1

for rank in ranks:
    als = ALS(maxIter=iterations, regParam=regularization_parameter, rank=rank,userCol="userID_index", itemCol="ISBN_index", ratingCol="rating")
    model = als.fit(training_df)
    predictions = model.transform(validation_df)
    new_predictions = predictions.filter(col('prediction') != np.nan)
    evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
    rmse = evaluator.evaluate(new_predictions)
    errors.append(rmse)

    print ('For rank %s the RMSE is %s' % (rank, rmse))
    if rmse < min_error:
        min_error = rmse
        best_rank = rank
print ('The best model was trained with rank %s' % best_rank)

Sekarang kita akan membangun CrossValidator untuk mengevaluasi rentang peringkat yang sama dengan konstanta regularisasi yang berbeda. Saya akan menggunakan  Sparks CrossValidator class. CrossValidator membutuhkan Estimator, satu set Estimator ParamMaps, dan Evaluator. Estimator yang akan kita gunakan adalah ALS, sama seperti di atas. Kami menggunakan ParamGridBuilder untuk membangun grid parameter untuk mencari. Dengan 2 nilai untuk als.regParam dan 8 nilai untuk als.rank, grid ini akan memiliki 2 x 8 = 16 pengaturan parameter untuk CrossValidator untuk dipilih.

In [ ]:
als = ALS(maxIter=iterations, regParam=regularization_parameter, rank=rank, userCol="user_id", itemCol="book_id", ratingCol="rating")
paramGrid = ParamGridBuilder() \
    .addGrid(als.regParam, [0.1, 0.01]) \
    .addGrid(als.rank, range(4, 12)) \
    .build()
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
crossval = CrossValidator(estimator=als,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=5)
cvModel = crossval.fit(training_df)

In [ ]:
cvModel_pred = cvModel.transform(validation_df)
cvModel_pred = cvModel_pred.filter(col('prediction') != np.nan)
rmse = evaluator.evaluate(cvModel_pred)
print ("the rmse for optimal grid parameters with cross validation is: {}".format(rmse))

the rmse for optimal grid parameters with cross validation is: 0.9405474411622875


Pencarian grid tidak bekerja sebaik yang di atas karena beberapa alasan. Saya akan menggunakan model akhir berikut.

In [ ]:
final_als = ALS(maxIter=10, regParam=0.1, rank=6, userCol="user_id", itemCol="book_id", ratingCol="rating")
final_model = final_als.fit(training_df)
final_pred = final_model.transform(validation_df)
final_pred = final_pred.filter(col('prediction') != np.nan)
rmse = evaluator.evaluate(final_pred)
print ("the rmse for optimal grid parameters with cross validation is: {}".format(rmse))

the rmse for optimal grid parameters with cross validation is: 0.9250463115776485


Sekarang mari kita ambil hyptothetical user yang telah memberi peringkat pada beberapa film. Saya akan mengeluarkan user_id acak dari kisaran id pengguna di dataset. Selanjutnya, saya mendapatkan semua peringkat mereka dan mengurutkan untuk menampilkan 10 film dengan peringkat teratas.

In [ ]:
np.random.seed(42)
user_id = np.random.choice(numUsers)

In [ ]:
new_user_ratings = ratings_df.filter(ratings_df.userId == user_id)
new_user_ratings.sort('rating', ascending=True).take(10) # top rated movies for this user

[Row(userId=102, movieId=540, rating=1.0, timestamp=835877588),
 Row(userId=102, movieId=204, rating=2.0, timestamp=835876471),
 Row(userId=102, movieId=173, rating=2.0, timestamp=835876239),
 Row(userId=102, movieId=329, rating=2.0, timestamp=835877081),
 Row(userId=102, movieId=163, rating=3.0, timestamp=835877307),
 Row(userId=102, movieId=23, rating=3.0, timestamp=835877570),
 Row(userId=102, movieId=21, rating=3.0, timestamp=835876107),
 Row(userId=102, movieId=172, rating=3.0, timestamp=835876471),
 Row(userId=102, movieId=39, rating=3.0, timestamp=835876151),
 Row(userId=102, movieId=165, rating=3.0, timestamp=835875790)]

In [ ]:
new_user_ratings.describe('rating').show()

+-------+------------------+
|summary|            rating|
+-------+------------------+
|  count|                56|
|   mean| 3.357142857142857|
| stddev|0.7960943623504901|
|    min|               1.0|
|    max|               5.0|
+-------+------------------+

